# Section B: Base Model (no smoothing)

In [ ]:
class NgramModel:
    def __init__(self, n=2):
        self.n = n
        self.ngram_counts = {}
        self.context_counts = {}
        self.vocabulary = set()

    def tokenize(self, text):
        import re
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        words = text.split()
        return words

    def train(self, text):
        words = self.tokenize(text)

        for word in words:
            self.vocabulary.add(word)

        padded = ['<START>'] * (self.n - 1) + words + ['<END>']

        for i in range(len(padded) - self.n + 1):
            ngram = tuple(padded[i:i + self.n])
            context = ngram[:-1]

            self.ngram_counts[ngram] = self.ngram_counts.get(ngram, 0) + 1
            self.context_counts[context] = self.context_counts.get(context, 0) + 1

    def probability(self, context, word):
        if isinstance(context, list):
            context = tuple(context)

        ngram = context + (word,)
        ngram_count = self.ngram_counts.get(ngram, 0)
        context_count = self.context_counts.get(context, 0)

        vocab_size = len(self.vocabulary)
        return (ngram_count + 1) / (context_count + vocab_size)

    def predict_next(self, context_words, top_k=5):
        context = context_words[-(self.n-1):]
        context = tuple(context)

        predictions = []
        for word in self.vocabulary:
            prob = self.probability(context, word)
            if prob > 0:
                predictions.append((word, prob))

        predictions.sort(key=lambda x: x[1], reverse=True)
        return predictions[:top_k]


if __name__ == "__main__":
    with open('/content/train_twi.txt', 'r', encoding='utf-8') as f:
        train_twi = f.read()

    # Create and train model
    model = NgramModel(n=2)  # bigram
    model.train(train_twi)

    print(f"Vocabulary size: {len(model.vocabulary)}")
    print(f"Unique bigrams: {len(model.ngram_counts)}")

    # Predict next word after "me din"
    predictions = model.predict_next(["me", "din"], top_k=5)
    print("\nTop predictions after 'me din':")
    for word, prob in predictions:
        print(f"  {word}: {prob:.3f}")

#Back Off Smoothing

In [ ]:
class NgramModel:
    def __init__(self, n=2):
        self.n = n
        self.ngram_counts = {}
        self.context_counts = {}
        self.vocabulary = set()

    def tokenize(self, text):
        import re
        text = text.lower()
        text = re.sub(r'[^ɛɔ\w\s]', '', text)
        words = text.split()
        return words

    def train(self, text):
        words = self.tokenize(text)

        for word in words:
            self.vocabulary.add(word)

        padded = ['<START>'] * (self.n - 1) + words + ['<END>']

        for i in range(len(padded) - self.n + 1):
            ngram = tuple(padded[i:i + self.n])
            context = ngram[:-1]

            self.ngram_counts[ngram] = self.ngram_counts.get(ngram, 0) + 1
            self.context_counts[context] = self.context_counts.get(context, 0) + 1

    def probability(self, context, word):
        if isinstance(context, list):
            context = tuple(context)

        ngram = context + (word,)
        ngram_count = self.ngram_counts.get(ngram, 0)
        context_count = self.context_counts.get(context, 0)

        # If we have seen this n-gram, use raw counts (no smoothing)
        if ngram_count > 0 and context_count > 0:
            return ngram_count / context_count

        # If we haven't seen it, back off
        elif len(context) > 0:
            return 0.4 * self.probability(context[1:], word)
        else:
            return 1.0 / len(self.vocabulary)

    def predict_next(self, context_words, top_k=5):
        context = context_words[-(self.n-1):]
        context = tuple(context)

        predictions = []
        for word in self.vocabulary:
            prob = self.probability(context, word)
            if prob > 0:
                predictions.append((word, prob))

        predictions.sort(key=lambda x: x[1], reverse=True)
        return predictions[:top_k]


if __name__ == "__main__":
    with open('/content/train_twi.txt', 'r', encoding='utf-8') as f:
        twi_text = f.read()

    model = NgramModel(n=2)
    model.train(twi_text)

    print(f"Vocabulary size: {len(model.vocabulary)}")
    print(f"Unique bigrams: {len(model.ngram_counts)}")

    # Predict next word after "me din"
    predictions = model.predict_next(["me", "din"], top_k=5)
    print("\nTop predictions after 'me din':")
    for word, prob in predictions:
        print(f"  {word}: {prob:.3f}")





##Updated Backoff (where each ngram model holds its own counts)

In [ ]:
import math

class NgramModelWithBackoff:
    def __init__(self, n=3, alpha=0.4):
        self.n = n
        self.alpha = alpha  #backoff penalty
        self.models = {} #storing models of different orders
        for i in range(1, n + 1):
            self.models[i] = {
                'ngram_counts': {},
                'context_counts': {},
                'vocabulary': set()
            }

    def tokenize(self, text):
        import re
        text = text.lower()
        text = re.sub(r'[^ɛɔ\w\s]', '', text)
        words = text.split()
        return words

    def train(self, text):
        words = self.tokenize(text)

        # training models of all orders
        for order in range(1, self.n + 1):
            vocab = self.models[order]['vocabulary']
            vocab.add('<END>')
            ngram_counts = self.models[order]['ngram_counts']
            context_counts = self.models[order]['context_counts']

            for word in words:
                vocab.add(word)

            padded = ['<START>'] * (order - 1) + words + ['<END>']

            for i in range(len(padded) - order + 1):
                ngram = tuple(padded[i:i + order])
                context = ngram[:-1] if order > 1 else ()

                ngram_counts[ngram] = ngram_counts.get(ngram, 0) + 1
                if order > 1:
                    context_counts[context] = context_counts.get(context, 0) + 1

    def probability(self, context, word, order=None):
        """
        Stupid backoff: if high-order n-gram not found, back off to lower order
        """
        if order is None:
            order = self.n

        if isinstance(context, list):
            context = tuple(context)

        #only relevant context for this order
        context = context[-(order-1):] if order > 1 else ()

        model = self.models[order]
        ngram = context + (word,)

        ngram_count = model['ngram_counts'].get(ngram, 0)

        if order == 1:
            # Unigram: P(word) = count(word) / total_words
            total_words = sum(model['ngram_counts'].values())
            if total_words == 0:
                return 1.0 / len(model['vocabulary']) if len(model['vocabulary']) > 0 else 1.0
            return ngram_count / total_words if ngram_count > 0 else 1.0 / total_words

        context_count = model['context_counts'].get(context, 0)

        if ngram_count > 0 and context_count > 0:
            return ngram_count / context_count
        elif order > 1:
            # Back off to shorter context with penalty
            return self.alpha * self.probability(context[1:] if len(context) > 0 else (), word, order - 1)
        else:
            #fallback to unigram
            return 1.0 / len(model['vocabulary']) if len(model['vocabulary']) > 0 else 1.0

    def perplexity(self, test_text):
        """
        Calculate perplexity on test text

        Lower perplexity = better model
        """
        words = self.tokenize(test_text)

        if len(words) == 0:
            return float('inf')

        #adding padding using <start>
        padded = ['<START>'] * (self.n - 1) + words + ['<END>']

        log_prob_sum = 0.0
        word_count = 0

        # Calculate probability for each word given its context
        for i in range(self.n - 1, len(padded)):
            context = tuple(padded[i - (self.n - 1):i])
            word = padded[i]

            prob = self.probability(context, word)
            if prob > 0:
                log_prob_sum += math.log2(prob)
                word_count += 1
            else:
                # If probability is 0, perplexity is infinite
                return float('inf')

        if word_count == 0:
            return float('inf')

        avg_log_prob = log_prob_sum / word_count
        perplexity = 2 ** (-avg_log_prob)

        return perplexity

    def predict_next(self, context_words, top_k=5):
        """Predict next word given context with top k approach"""
        context = context_words[-(self.n-1):]
        context = tuple(context)

        predictions = []
        vocab = self.models[self.n]['vocabulary']

        for word in vocab:
            prob = self.probability(context, word)
            if prob > 0:
                predictions.append((word, prob))

        predictions.sort(key=lambda x: x[1], reverse=True)
        return predictions[:top_k]

    def sample_next_word(self, context, temperature=1.0):
      """
      Sample next word according to probability distribution (weighted sampling)

      Args:
          context: Previous words
          temperature: Controls randomness (0.5=conservative, 1.0=normal, 2.0=creative)

      Returns:
          Sampled word
      """
      import random
      import math

      context = tuple(context[-(self.n-1):])
      vocab = self.models[self.n]['vocabulary']

      # Get probabilities for all words
      probs = []
      words = []

      for word in vocab:
          prob = self.probability(context, word)
          if prob > 0:
              words.append(word)
              probs.append(prob)

      if len(words) == 0:
          return '<END>'

      # Apply temperature
      if temperature != 1.0:
          # Adjust probabilities: p_i = p_i^(1/T) / sum(p_j^(1/T))
          adjusted_probs = [p ** (1.0 / temperature) for p in probs]
          total = sum(adjusted_probs)
          probs = [p / total for p in adjusted_probs]

      # Sample weighted random choice
      rand = random.random()
      cumulative = 0.0

      for word, prob in zip(words, probs):
          cumulative += prob
          if rand <= cumulative:
              return word

      return words[-1]

    def sample_generate(self, start_context, max_length=20, temperature=1.0):
        """
        Generate sequence by sampling
        """
        sequence = start_context.copy()

        for _ in range(max_length):
            context = sequence[-(self.n-1):]
            next_word = self.sample_next_word(context, temperature)

            if next_word == '<END>':
                break

            sequence.append(next_word)

        return sequence



if __name__ == "__main__":
    # Training data
    with open('/content/train_twi.txt', 'r', encoding='utf-8') as f:
        train_text = f.read()

    # Test data
    with open('/content/test_twi.txt', 'r', encoding='utf-8') as f:
        test_text = f.read()
    # Val data
    with open('/content/val_twi.txt', 'r', encoding='utf-8') as f:
        val_text = f.read()

    # Train models of different orders
    print("Training models...\n")

    bigram_model = NgramModelWithBackoff(n=2, alpha=0.4)
    bigram_model.train(train_text)

    trigram_model = NgramModelWithBackoff(n=3, alpha=0.4)
    trigram_model.train(train_text)

    # Evaluate on test data
    print("=" * 50)
    print("PERPLEXITY EVALUATION")
    print("=" * 50)

    bigram_perplexity = bigram_model.perplexity(test_text)
    trigram_perplexity = trigram_model.perplexity(test_text)

    print(f"Bigram model perplexity:  {bigram_perplexity:.2f}")
    print(f"Trigram model perplexity: {trigram_perplexity:.2f}")

    if bigram_perplexity < trigram_perplexity:
        print("\n Bigram model is better (lower perplexity)")
    else:
        print("\n Trigram model is better (lower perplexity)")

    print("\n" + "=" * 50)
    print("SAMPLE PREDICTIONS")
    print("=" * 50)
    print("\nBigram predictions:")
    for i in range(3):
        seq = bigram_model.sample_generate(["me", "pɛ"], temperature=0.8)
        print(f"Sample {i+1}: {' '.join(seq)}")
    print("\nTrigram predictions:")
    for i in range(3):
        seq = trigram_model.sample_generate(["me", "pɛ"], temperature=0.8)
        print(f"Sample {i+1}: {' '.join(seq)}")

# Using top k decoding
    # test_context = ["me", "pɛ"]
    # print(f"\nContext: {' '.join(test_context)}")
    # print("\nBigram predictions:")
    # for word, prob in bigram_model.sample_generate(test_context, top_k=5):
    #     print(f"  {word}: {prob:.4f}")

    # print("\nTrigram predictions:")
    # for word, prob in trigram_model.sample_generate(test_context, top_k=5):
    #     print(f"  {word}: {prob:.4f}")

## Validation & Evaluation Experiments (with back off alpha)

In [ ]:
def train_with_validation(model_class):
    """
    Uses pre-split train/validation/test data from files.
    """
    # Load pre-split data
    with open('/content/train_twi.txt', 'r', encoding='utf-8') as f:
        train_text = f.read()
    with open('/content/val_twi.txt', 'r', encoding='utf-8') as f:
        val_text = f.read()
    with open('/content/test_twi.txt', 'r', encoding='utf-8') as f:
        test_text = f.read()

    # For reporting purposes, tokenize and count sentences for each set
    # (using newline as a proxy for sentences in this context)
    train_sentences = [s.strip() for s in train_text.split('\n') if s.strip()]
    val_sentences = [s.strip() for s in val_text.split('\n') if s.strip()]
    test_sentences = [s.strip() for s in test_text.split('\n') if s.strip()]

    print(f"Train: {len(train_sentences)} sentences")
    print(f"Validation: {len(val_sentences)} sentences")
    print(f"Test: {len(test_sentences)} sentences\n")

    # Try different hyperparameters
    best_model = None
    best_val_pp = float('inf')
    results = []

    for n_val in [2, 3, 4]:
        for alpha in [0.5, 0.6, 0.7, 0.8, 0.9]:
            model = model_class(n=n_val, alpha=alpha)
            model.train(train_text)

            val_pp = model.perplexity(val_text)
            results.append((n_val, alpha, val_pp))

            print(f"n={n_val}, alpha={alpha:.1f}: validation perplexity={val_pp:.2f}")

            if val_pp < best_val_pp:
                best_val_pp = val_pp
                best_model = (n_val, alpha, model)

    # Test best model
    print(f"\n{'='*50}")
    print(f"Best model on validation set:")
    print(f"  n={best_model[0]}, alpha={best_model[1]}")
    print(f"  Validation perplexity: {best_val_pp:.2f}")

    test_pp = best_model[2].perplexity(test_text)
    print(f"  Test perplexity: {test_pp:.2f}")
    print(f"{'='*50}")

    return best_model, results

# Usage
best_model, results = train_with_validation(NgramModelWithBackoff)

#Analysing Data

In [ ]:
def analyze_data_sparsity(model):
    """
    Understand why predictions are uncertain
    """
    print("="*60)
    print("DATA SPARSITY ANALYSIS")
    print("="*60)

    for order in range(1, model.n + 1):
        ngram_counts = model.models[order]['ngram_counts']
        context_counts = model.models[order]['context_counts']
        vocab = model.models[order]['vocabulary']

        # Count how many n-grams appear only once
        singletons = sum(1 for count in ngram_counts.values() if count == 1)
        total_ngrams = len(ngram_counts)

        # Calculate coverage
        total_count = sum(ngram_counts.values())
        singleton_mass = sum(count for count in ngram_counts.values() if count == 1)

        print(f"\n{order}-grams:")
        print(f"  Vocabulary size: {len(vocab)}")
        print(f"  Unique {order}-grams: {total_ngrams}")
        print(f"  Singletons (seen once): {singletons} ({100*singletons/total_ngrams:.1f}%)")
        print(f"  Probability mass in singletons: {100*singleton_mass/total_count:.1f}%")

        if order == model.n:
            # Check how many contexts have multiple continuations
            multi_continuation = 0
            for context in context_counts:
                continuations = sum(1 for ng in ngram_counts if ng[:-1] == context)
                if continuations > 1:
                    multi_continuation += 1

            print(f"  Contexts with multiple continuations: {multi_continuation}/{len(context_counts)}")

analyze_data_sparsity(best_model[2])

#Testing Interpolation

In [ ]:
class InterpolatedNgramModel:
    def __init__(self, n=3):
        self.n = n
        self.models = {}
        for i in range(1, n + 1):
            self.models[i] = {
                'ngram_counts': {},
                'context_counts': {},
                'vocabulary': set()
            }

    def tokenize(self, text):
        import re
        text = text.lower()
        text = re.sub(r'[^ɛɔ\w\s]', '', text)
        words = text.split()
        return words

    def train(self, text):
        words = self.tokenize(text)

        for order in range(1, self.n + 1):
            vocab = self.models[order]['vocabulary']
            ngram_counts = self.models[order]['ngram_counts']
            context_counts = self.models[order]['context_counts']

            for word in words:
                vocab.add(word)

            padded = ['<START>'] * (order - 1) + words + ['<END>']

            for i in range(len(padded) - order + 1):
                ngram = tuple(padded[i:i + order])
                context = ngram[:-1] if order > 1 else ()

                ngram_counts[ngram] = ngram_counts.get(ngram, 0) + 1
                if order > 1:
                    context_counts[context] = context_counts.get(context, 0) + 1

    def probability(self, context, word, lambdas=None):
        """
        Linear interpolation: mix all n-gram orders
        """
        if lambdas is None:
            lambdas = [0.5, 0.3, 0.2]  # Default for trigram

        if isinstance(context, list):
            context = tuple(context)

        # Make sure lambdas sum to 1.0
        lambda_sum = sum(lambdas)
        lambdas = [l / lambda_sum for l in lambdas]

        total_prob = 0.0

        # Calculate probability from each order
        for order in range(1, self.n + 1):
            if order - 1 >= len(lambdas):
                break

            # Extract relevant context for this order
            if order == 1:
                relevant_context = ()
            else:
                # Take last (order-1) words from context
                relevant_context = context[-(order-1):] if len(context) >= order - 1 else context

            model = self.models[order]
            ngram = relevant_context + (word,)

            ngram_count = model['ngram_counts'].get(ngram, 0)

            if order == 1:
                # Unigram: P(word) = count(word) / total_words
                total_words = sum(model['ngram_counts'].values())
                if total_words > 0:
                    prob = ngram_count / total_words
                else:
                    prob = 0
            else:
                # Higher order: P(word|context) = count(context,word) / count(context)
                context_count = model['context_counts'].get(relevant_context, 0)
                if context_count > 0:
                    prob = ngram_count / context_count
                else:
                    prob = 0

            # Add weighted probability
            total_prob += lambdas[order - 1] * prob

        if total_prob == 0:
            return 1.0 / len(self.models[1]['vocabulary'])

        return total_prob

    def perplexity(self, test_text):
        import math
        words = self.tokenize(test_text)

        if len(words) == 0:
            return float('inf')

        padded = ['<START>'] * (self.n - 1) + words + ['<END>']

        log_prob_sum = 0.0
        word_count = 0

        for i in range(self.n - 1, len(padded)):
            context = tuple(padded[i - (self.n - 1):i])
            word = padded[i]

            prob = self.probability(context, word)

            if prob > 0:
                log_prob_sum += math.log2(prob)
                word_count += 1
            else:
                return float('inf')

        if word_count == 0:
            return float('inf')

        avg_log_prob = log_prob_sum / word_count
        perplexity = 2 ** (-avg_log_prob)

        return perplexity

    def predict_next(self, context_words, top_k=5):
        context = context_words[-(self.n-1):]
        context = tuple(context)

        predictions = []
        vocab = self.models[self.n]['vocabulary']

        for word in vocab:
            prob = self.probability(context, word)
            if prob > 0:
                predictions.append((word, prob))

        predictions.sort(key=lambda x: x[1], reverse=True)
        return predictions[:top_k]

for lambdas in [[0.6, 0.3, 0.1], [0.5, 0.3, 0.2], [0.4, 0.4, 0.2]]:
    model = InterpolatedNgramModel(n=3)
    model.train(train_text)
    val_pp = model.perplexity(val_text)
    print(f"lambdas={lambdas}: {val_pp:.2f}")